# Dot Product 구현

### Use CPU

In [1]:
import numpy as np

a_mat = np.random.randint(5, size=[10, 10])
b_mat = np.random.randint(5, size=[10, 10])
ret_mat_cpu = np.dot(a_mat, b_mat)

print("a_mat =")
print(a_mat)
print("\nb_mat =")
print(b_mat)
print("\nret_mat =")
print(ret_mat_cpu)

a_mat =
[[1 2 2 3 2 2 2 3 4 4]
 [2 3 4 4 2 3 0 4 2 2]
 [4 2 0 3 3 3 0 3 2 2]
 [0 2 3 0 2 4 3 4 3 2]
 [4 1 3 4 1 4 3 0 4 1]
 [4 1 2 3 1 0 2 0 0 0]
 [2 3 4 1 0 1 3 0 4 2]
 [2 2 1 0 4 4 2 1 4 1]
 [4 0 0 3 0 1 3 3 4 4]
 [2 4 4 2 3 2 4 2 1 3]]

b_mat =
[[4 2 0 2 3 1 4 2 4 2]
 [0 3 4 3 1 3 1 2 1 2]
 [0 4 2 3 1 4 4 0 4 1]
 [4 1 2 4 3 0 0 4 3 1]
 [4 0 0 2 1 3 3 2 0 2]
 [3 1 3 2 2 4 0 0 3 0]
 [3 4 1 4 3 2 0 0 3 4]
 [3 4 3 4 1 2 3 0 4 2]
 [4 0 4 2 0 4 0 0 2 3]
 [1 3 3 3 2 3 1 0 2 0]]

ret_mat =
[[65 53 63 74 39 67 33 22 63 41]
 [63 58 63 77 41 67 47 30 72 36]
 [68 38 46 60 39 51 38 30 56 33]
 [55 56 59 67 32 72 34  8 61 40]
 [74 46 52 69 48 63 33 28 72 43]
 [38 30 16 39 31 22 28 24 40 25]
 [42 49 50 57 31 59 29 14 54 39]
 [62 33 46 52 30 65 30 16 46 39]
 [69 48 49 66 43 48 29 20 65 41]
 [59 69 57 81 47 72 46 26 68 47]]


### Use GPU

In [2]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda import driver, compiler

In [3]:
# Kernel code
kernel_code = """
__global__ void dot(int* in_arr1, int* in_arr2, int* ret_arr)
{
    // TODO
}
"""

# Compile the kernel code
mod = compiler.SourceModule(kernel_code)

# Get kernel function
dot_func = mod.get_function("dot")

# Run & Print
ret_mat_gpu = np.zeros_like(a_mat)
dot_func(cuda.In(a_mat), cuda.In(b_mat), cuda.Out(ret_mat_gpu), block=(5, 5, 1), grid=(2, 2))

print("\nCPU로 계산 =")
print(ret_mat_cpu)
print("\nGPU로 계산 =")
print(ret_mat_gpu)


CPU로 계산 =
[[65 53 63 74 39 67 33 22 63 41]
 [63 58 63 77 41 67 47 30 72 36]
 [68 38 46 60 39 51 38 30 56 33]
 [55 56 59 67 32 72 34  8 61 40]
 [74 46 52 69 48 63 33 28 72 43]
 [38 30 16 39 31 22 28 24 40 25]
 [42 49 50 57 31 59 29 14 54 39]
 [62 33 46 52 30 65 30 16 46 39]
 [69 48 49 66 43 48 29 20 65 41]
 [59 69 57 81 47 72 46 26 68 47]]

GPU로 계산 =
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
